In [ ]:
'''
cross_val_score() 함수
 fold set 추출, 학습/예측, 평가를
 한 번에 수행하는 점수
 
GridSearchCV
 교차 검증과 최적 하이퍼 파라미터 튜닝을 한번에 함
                   ㄴ Classifier(분류) / Regressor(화가)알고리즘에서 사용함
  하이퍼 파라미터 순차적으로 입력하면서
  최적의 파라미터 도출
'''

#### crossvalue

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

'''
scoring= "accuracy" <-- 성능 지표를 정확도로함
cv=3               <-- 교차검증 세트를 3개로 함
'''
#교차 검증별 정확도를 리스트로 반환함
score = cross_val_score(dt_clf, data, label, scoring="accuracy",cv=3)

print(f"교차 검증별 정확도 : {np.round(score,4)}")
print(f"평균 검증 정확도 : {np.round(np.mean(score),4)}")

교차 검증별 정확도 : [0.98 0.94 0.98]
평균 검증 정확도 : 0.9667


### GridSearchCV

In [31]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 데이터 세트 로딩하고 학습데이터와 테스트데이터 분리하기
iris = load_iris()
X_train, X_test, y_train, y_test = \
  train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)
dt_clf = DecisionTreeClassifier()

'''
parameter 들을 dictionary로 설정하기
 ㄴ GridSearchCV 수행할 때 지정한 parameter 값들로 반복하면서 수행하면서
    최적의 parameter 값을 찾음
max_depth, min_samples_split : hyper parameter 이름
[1,2,3], [2,3] : 순차적으로 수행할 때의 hyper parameter 값들
'''
# r param_grid
parameters = {"max_depth":[1,2,3], "min_samples_split":[2,3]}

In [35]:
import pandas as pd

'''
cv=3 : hyper parameter 들을 3개 train/ test set fold 로 나눔
refit = True (default) - True 인 경우 가장 좋은 parameter 설정으로 재학습시킴
GridSearchCV 생성자의 주요 parameter
estimator 모델 (classification, regression)
param_grid : {사용되는 parameter 명 : 값}, <-- dictionary 로 설정함
cv : 교차 검증할때 분할하는 학습/테스트 세트의 개수
'''
grid_dt_clf = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True, return_train_score=True)

# iris train data 로 paramt_grid값들로 순차적으로 학습/평가 수행하기
grid_dt_clf.fit(X_train, y_train)

'''
# GridSearchCV 학습 결과는 cv_results_ 라는 dictionary에 저장도미
이것을 DataFrame으로 변환해서 사용함
'''
scores_df = pd.DataFrame(grid_dt_clf.cv_results_)
scores_df[["params","mean_test_score","rank_test_score",
          "split0_test_score", "split1_test_score", "split2_test_score"]
         ]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [36]:
print("grid_dt_clf.cv_results_ :\n",grid_dt_clf.cv_results_,sep="")

grid_dt_clf.cv_results_ :
{'mean_fit_time': array([0.00066519, 0.0006663 , 0.00101002, 0.00033267, 0.00099182,
       0.00099715]), 'std_fit_time': array([4.70358870e-04, 4.71151362e-04, 1.46743144e-05, 4.70471221e-04,
       6.57562879e-06, 4.49566384e-07]), 'mean_score_time': array([0.0006698 , 0.00099627, 0.00065343, 0.00066495, 0.00033283,
       0.00033259]), 'std_score_time': array([4.73661751e-04, 2.59232623e-06, 4.62272469e-04, 4.70190252e-04,
       4.70696004e-04, 4.70358829e-04]), 'param_max_depth': masked_array(data=[1, 1, 2, 2, 3, 3],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 1, 'min_samples_split': 2}, {'max_depth': 1, 'min_samples_split': 3}, {'max_depth': 2, 'min_samples_split': 2}, {'max_depth': 2, 'mi

In [39]:
print(f"grid_dt_clf.cv 최적파라미터 :\n",grid_dt_clf.best_params_)
print(f"gridsearch 최고정확도 :,{grid_dt_clf.best_score_:4f}")

grid_dt_clf.cv 최적파라미터 :
 {'max_depth': 3, 'min_samples_split': 2}
gridsearch 최고정확도 :,0.975000


In [38]:
print("--- 테스트 데이터 세트의 정확도 구하기 ---")
'''
refit = True 로 설정된 GridSearchCV 객체의 fit() 메소드가 호출된 후,
(학습이 완료된) Estimator(모델)를 내포하고 있으므로
predict()를 호출해서 예측도 수행할 수 있음
'''
pred = grid_dt_clf.predict(X_test)
print(f"테스트 데이터 세트의 정확도 : {accuracy_score(y_test,pred):.4f}")

--- 테스트 데이터 세트의 정확도 구하기 ---
테스트 데이터 세트의 정확도 : 0.9667


In [40]:
# GirdSearchCV 의 refit 으로 이미 학습된 estimator 를 반환함 : best_estimator 속성
estimator = grid_dt_clf.best_estimator_
'''
GridSearchCV의 best_estimator_는 이미 최적 hyper parameter로 학습된 상태임
이미 최적 hyper parameter 로 학습된 상태임
'''
pred = estimator.predict(X_test)
print(f"테스트 데이터 세트의 정확도 :{accuracy_score(y_test,pred):.4f}")

테스트 데이터 세트의 정확도 :0.9667
